<a href="https://colab.research.google.com/github/RobertBWeidlich/Colab_Notebooks/blob/main/Budget_2025_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#
# 1. manually export data from XLS to CSV file:
# (in Excel)
#   Excel -> Open sheet to be analyzed -> File -> Export ->
#       Change File Type -> CSV


# move csv data from local Windows host to local linux server
# for analysis

from google.colab import files
uploaded = files.upload()
print(uploaded)
file_name = next(iter(uploaded))
#data = uploaded[file_name]
#print(data)

In [ ]:
"""
type(uploaded)
import pprint as pp
#pp.pprint(uploaded)
kk = uploaded.keys()
print(type(kk))
print(kk)
file_name = next(iter(uploaded))
print(file_name)
"""

In [ ]:
# 2. load data into pandas and print it out.
import pandas as pd

dtype_spec = {
    "Description": str,
    "Category": str,
    "Source Account": str,
    "Check #": "Int64",     # Use nullable integer type
    "Priority": "Int64",
    "Amount": float
}

df = pd.read_csv(file_name,
                 encoding="cp1252",
                 usecols=range(8))

#df = pd.read_csv(file_name, dtype=dtype_spec, parse_dates=["Timestamp"])

print(df.dtypes)

# drop all rows where all cells are NaN
df = df.dropna(how='all')

# set "Check #" column to integer
# openai - doesn't work
#df["Check #"] = pd.to_numeric(df["Check #"], errors="coerce").astype("Int64")
# gemini - doesn't work
#df["Check #"] = pd.to_numeric(df["Check #"], errors="coerce").astype("Int64")

# this does not work with colab
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#  print(df)
# this DOES work with colab
#print(df.to_string())
print(df.head(20).to_string())


In [ ]:
# 3. sort by the "Category" column to check for variations category names
df_sorted = df.sort_values(by="Category", ascending=True)
#print(df_sorted.head(20).to_string())
print(df_sorted.head(5000).to_string())

In [ ]:
# 4. now sort by the "Source Account" column to check for variations
df_sorted = df.sort_values(by="Source Account", ascending=True)
#print(df_sorted.head(20).to_string())
print(df_sorted.head(5000).to_string())

In [ ]:

# take a list of numbers in various formats ("1", "1.00", "$1.00")
# and return sum in currency format
# NOTE: don't use this; instead use:
#   [68.93, 12.50, 100.25]
#
def sum_of_list_currency_format(cur_list):
  sum = 0.0
  for cur in cur_list:
    #print(type(cur))
    if type(cur) == str:
      cur = cur.replace('$', '').replace(',', '')
    sum += float(cur)
  return(round(sum, 2))

test_cur_list = ["13.00", "$54.98", 1]
sum_of_list_currency_format(test_cur_list)

In [ ]:
# 5. Analysis: for each "Category" column, find sum of "Amount" column
#
# 5A. set some columns to lower case
#print(type(df['Category'].astype(str).str.lower()))
df['Category'] = df['Category'].astype(str).str.lower()

# 5B. get list of unique categories and unique major-categories.
#     for a category "abc-345", "abc" is the major category.
#
unique_categories = sorted(df['Category'].unique().tolist())
print("unique_categories:")
print(unique_categories)
print("unique_major_categories:")
unique_major_categories = sorted(set(item.split('-')[0] for item in unique_categories))
print(unique_major_categories)

In [ ]:
# 6. for each category, print all records matching that category, along with the sum:
acc_cat_sums = [] # accumulate data, implemented as list of tuples.
                  # each tuple contains a category and the sum
                  # off all entries in that category
num_months = 3.0  # how many months of data

sum_per_month = 0.0
agg_sum = 0.0
agg_sum_per_month = 0.0
for cat in unique_categories:
  print()
  print(f"category: {cat}")
  print("========")
  filtered_df = df[df['Category'].str.lower() == cat]
  print(filtered_df.head(500).to_string())
  # get sum of "Amount" column using sum_of_list_currency_format()
  amount_list = filtered_df["Amount"].tolist()
  #print(amount_list)
  sum = sum_of_list_currency_format(amount_list)
  sum_per_month = sum / num_months
  print(f"sum: {sum}")
  print(f"sum_per_month: {sum_per_month}")
  acc_cat_sums.append((cat, sum, sum_per_month))
  agg_sum += sum
  agg_sum_per_month += sum_per_month
  print("========")
print("########")
print(acc_cat_sums)
print(agg_sum)
print(agg_sum_per_month)
print("########")
# print summary
print()
for ac_sum in acc_cat_sums:


  v1 = ac_sum[0]
  v1 = f"{v1:<20}"
  v2 = ac_sum[1]
  v2 = f"{v2:12.2f}"
  v3 = ac_sum[2]
  v3 = f"{v3:12.2f}"
  #print(ac_sum)
  print(f"  {v1} {v2} {v3}")
